# Ontology Tools Experiment Notebook
* This notebook contains experiments using the Ontology Tools.

* Demo `transform_radlex()` function
    1. Accessing the "ontologies" database in MongoDB.
    2. Demo the `transform_radlex()` function on 1 JSON document.
    3. Demo the `transform_radlex()` function on multiple JSON document.

* Demo JSON dump using `batch_write_mongoDB` function
    1. Demo 100 JSON documents dump.


* Final Script in notebook
    1. Run 100 document batches => write back to `radlex` collection in `ontologies` database within MongoDB.


In [12]:
# import local modules
# import MongoDB modules
import motor.motor_asyncio
from dotenv import dotenv_values
from openimagingdatamodel.ontology_tools import RadLexConcept, transform_radlex

In [13]:
# establish connection to MongoDB
config = dotenv_values(".env")
client = motor.motor_asyncio.AsyncIOMotorClient(config["ATLAS_DSN"])
db = client["ontologies"]
collection = db["RadLex"]
# Get the count of documents in the collection to confirm that the data was loaded
count = await collection.count_documents({})
print(f"Count of documents in the collection: {count}")

Count of documents in the collection: 46761


In [14]:
# Access the ontologies database and "RadLex" collection
doc = await collection.find_one()

In [15]:
print(doc)

{'_id': ObjectId('65f84ed1f80fad5323c79c1f'), 'Class ID': 'http://radlex.org/RID/RID35591', 'Preferred Label': 'string-of-pearls sign of bowel', 'Definitions': 'Oblique or horizontal row of air bubbles visible on abdominal radiograph; almost always indicates small bowel obstruction; air is trapped between valvulae conniventes along the superior wall of the intestine.', 'Obsolete': False, 'Parents': 'http://radlex.org/RID/RID29023', 'http://data': {'bioontology': {'org/metadata/prefixIRI': 'RID35591'}}, 'http://radlex': {'org/RID/Anatomical_Site': 'http://radlex.org/RID/RID132', 'org/RID/Comment': 'http://radiology.rsna.org/cgi/content/full/214/1/157', 'org/RID/Definition': 'Oblique or horizontal row of air bubbles visible on abdominal radiograph; almost always indicates small bowel obstruction; air is trapped between valvulae conniventes along the superior wall of the intestine.', 'org/RID/May_Be_Caused_By': 'http://radlex.org/RID/RID4962', 'org/RID/Preferred_name': 'string-of-pearls s

### Demo `transform_radlex()` function before writing to MongoDB

#### Demo function on 1 document

In [16]:
t_doc = transform_radlex(doc)
print(t_doc.model_dump_json(indent=2))

{
  "id": "RID35591",
  "preferred_label": "string-of-pearls sign of bowel",
  "synonyms": null,
  "parent": "RID29023",
  "definition": "Oblique or horizontal row of air bubbles visible on abdominal radiograph; almost always indicates small bowel obstruction; air is trapped between valvulae conniventes along the superior wall of the intestine.",
  "radlex_properties": {
    "anatomicalSite": "RID132",
    "comment": "157",
    "mayBeCausedBy": "RID4962",
    "relatedModality": "RID10345",
    "source": "Radiology 2000; 214:157-158"
  }
}


#### Demo function on multiple documents

In [17]:
# Check multiple documents
docs = await collection.find({}).to_list(length=10)
for doc in docs:
    t_doc = transform_radlex(doc)
    print(t_doc.model_dump_json(indent=2))

{
  "id": "RID35591",
  "preferred_label": "string-of-pearls sign of bowel",
  "synonyms": null,
  "parent": "RID29023",
  "definition": "Oblique or horizontal row of air bubbles visible on abdominal radiograph; almost always indicates small bowel obstruction; air is trapped between valvulae conniventes along the superior wall of the intestine.",
  "radlex_properties": {
    "anatomicalSite": "RID132",
    "comment": "157",
    "mayBeCausedBy": "RID4962",
    "relatedModality": "RID10345",
    "source": "Radiology 2000; 214:157-158"
  }
}
{
  "id": "RID45335",
  "preferred_label": "peripheral segment of nerve to left sternocleidomastoid",
  "synonyms": null,
  "parent": "RID45333",
  "definition": "",
  "radlex_properties": {
    "fmaid": "288509",
    "hasRegionalPart": "RID22021",
    "regionalPartOf": "RID27715"
  }
}
{
  "id": "RID30121",
  "preferred_label": "surface of cardia of stomach",
  "synonyms": null,
  "parent": "RID30094",
  "definition": "",
  "radlex_properties": {
   

#### Code to check if "|" was removed from JSON docs

In [18]:
# check multiple docs if any instance of not transformed "|"
import json

docs = await collection.find({}).to_list(length=10)
for doc in docs:
    t_doc = transform_radlex(doc)
    t_doc_json = json.dumps(t_doc.dict())
    if "|" in t_doc_json:
        print(f"Error in transforming document {t_doc.id}: '|' character found.")

# Code to write transformed RadLex to new 'radlex' collection in MongoDB


In [19]:
doc = await collection.find_one()

print(doc)

{'_id': ObjectId('65f84ed1f80fad5323c79c1f'), 'Class ID': 'http://radlex.org/RID/RID35591', 'Preferred Label': 'string-of-pearls sign of bowel', 'Definitions': 'Oblique or horizontal row of air bubbles visible on abdominal radiograph; almost always indicates small bowel obstruction; air is trapped between valvulae conniventes along the superior wall of the intestine.', 'Obsolete': False, 'Parents': 'http://radlex.org/RID/RID29023', 'http://data': {'bioontology': {'org/metadata/prefixIRI': 'RID35591'}}, 'http://radlex': {'org/RID/Anatomical_Site': 'http://radlex.org/RID/RID132', 'org/RID/Comment': 'http://radiology.rsna.org/cgi/content/full/214/1/157', 'org/RID/Definition': 'Oblique or horizontal row of air bubbles visible on abdominal radiograph; almost always indicates small bowel obstruction; air is trapped between valvulae conniventes along the superior wall of the intestine.', 'org/RID/May_Be_Caused_By': 'http://radlex.org/RID/RID4962', 'org/RID/Preferred_name': 'string-of-pearls s

## Demo fetch docs from 'RadLex' collection in batches of 100 => demo JSON dumps

#### This demos the function

In [20]:
async def fetch_and_demo_docs():
    # Fetch 10 documents
    docs = await collection.find({}).to_list(length=100)

    # Iterate through the documents
    for doc in docs:
        # Transform the document
        transformed_doc = transform_radlex(doc)

        # Print JSON dump of the transformed document
        print(transformed_doc.model_dump_json(indent=2, by_alias=True, exclude_none=True))

# Run the async function
await fetch_and_demo_docs()


{
  "_id": "RID35591",
  "preferredLabel": "string-of-pearls sign of bowel",
  "parent": "RID29023",
  "definition": "Oblique or horizontal row of air bubbles visible on abdominal radiograph; almost always indicates small bowel obstruction; air is trapped between valvulae conniventes along the superior wall of the intestine.",
  "radlexProperties": {
    "anatomicalSite": "RID132",
    "comment": "157",
    "mayBeCausedBy": "RID4962",
    "relatedModality": "RID10345",
    "source": "Radiology 2000; 214:157-158"
  }
}
{
  "_id": "RID45335",
  "preferredLabel": "peripheral segment of nerve to left sternocleidomastoid",
  "parent": "RID45333",
  "definition": "",
  "radlexProperties": {
    "fmaid": "288509",
    "hasRegionalPart": "RID22021",
    "regionalPartOf": "RID27715"
  }
}
{
  "_id": "RID30121",
  "preferredLabel": "surface of cardia of stomach",
  "parent": "RID30094",
  "definition": "",
  "radlexProperties": {
    "fmaid": "17490"
  }
}
{
  "_id": "RID38411",
  "preferredLabe

### Now run these scripts to access the new 'radlex' collection within the 'ontologies' database and then run batch script

In [8]:
# establish connection to MongoDB
config = dotenv_values(".env")
client = motor.motor_asyncio.AsyncIOMotorClient(config["ATLAS_DSN"])
db = client["ontologies"]


##### change to the new collection "radlex"
collection_new = db["radLex"]



# Get the count of documents in the collection to confirm that the data was loaded - it should be zero before we load the data
count = await collection.count_documents({})
print(f"Count of documents in the collection: {count}")

Count of documents in the collection: 0


In [ ]:
# import the batch_transform_and_write_to_db function function
from openimagingdatamodel.ontology_tools import batch_transform_and_write_to_db


# Final Script to Write Transformed Documents to MongoDB `radlex` collection in `ontologies` database

In [ ]:
async def fetch_transform_and_write_docs():
    # Fetch documents in batches of 100
    cursor = collection.find({}).batch_size(100)

    batch = []
    async for doc in cursor:
        # Transform the document
        transformed_doc = transform_radlex(doc)

        # Print JSON dump of the transformed document (demo)
        print(transformed_doc.model_dump_json(indent=2))

        # Add transformed document to the batch
        batch.append(transformed_doc.model_dump())

        # If the batch size reaches 100, write it to the database
        if len(batch) == 100:
            batch_transform_and_write_to_db(collection, batch)
            batch = []

    # Write any remaining documents in the last batch
    if batch:
        batch_transform_and_write_to_db(collection, batch)

# Run the async function
await fetch_transform_and_write_docs()
